# **HOUSE PRICE PREDICTION**

Sử dụng các mô hình đã học hoặc nâng cao để áp dụng vào bài toán dự đoán giá nhà

In [41]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [42]:
train_df = pd.read_csv('./preprocessing_data/train.csv')
train_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,4,3,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,3,3,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,4,3,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,3,3,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,4,3,...,0,0,0,1,0,0,0,0,1,0


In [43]:
test_df = pd.read_csv('./preprocessing_data/test.csv')
test_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20,80.0,11622,5,6,1961,1961,0.0,3,3,...,0,0,0,1,0,0,0,0,1,0
1,20,81.0,14267,6,6,1958,1958,108.0,3,3,...,0,0,0,1,0,0,0,0,1,0
2,60,74.0,13830,5,5,1997,1998,0.0,3,3,...,0,0,0,1,0,0,0,0,1,0
3,60,78.0,9978,6,6,1998,1998,20.0,3,3,...,0,0,0,1,0,0,0,0,1,0
4,120,43.0,5005,8,5,1992,1992,0.0,4,3,...,0,0,0,1,0,0,0,0,1,0


## Xử lý dữ liệu huấn luyện

In [44]:
# Khai báo thư viện cần thiết
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Tách bộ dữ liệu từ tập `train` thành tập `train` nhỏ hơn và tập `valid`

In [45]:
X = np.array(train_df.drop(['SalePrice'], axis = 1).values)
y = np.array(train_df['SalePrice'].values)

In [46]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, random_state=42)
X_test = np.array(test_df.values)

Chuẩn hoá dữ liệu từ tập `train` để quá trình huấn luyện đạt kết quả tốt hơn

In [47]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_valid = sc_X.transform(X_valid)
X_test = sc_X.transform(X_test)

## 1. RANDOM FOREST

In [48]:
from sklearn.ensemble import RandomForestClassifier

### Không dùng parameter và đánh giá độ chính xác mô hình

In [49]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [50]:
y_pred_rf = rf.predict(X_valid)
y_train_pred_rf = rf.predict(X_train)

In [51]:
acc_valid = metrics.accuracy_score(y_valid, y_pred_rf)
print(f"Accuracy valid split = {acc_valid}")

Accuracy train split = 1.0
Accuracy valid split = 0.003424657534246575


### Tìm parameter tốt nhất và đánh giá mô hình

#### Tìm parameter tốt nhất

In [53]:
param_grid = {
    'n_estimators': [10, 25, 50],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

In [54]:
grid_search = GridSearchCV(RandomForestClassifier(), param_grid = param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_)

RandomForestClassifier(max_depth=3, max_leaf_nodes=6, n_estimators=25)


In [55]:
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

0.013 (+/-0.009) for {'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 3, 'n_estimators': 10}
0.012 (+/-0.003) for {'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 3, 'n_estimators': 25}
0.015 (+/-0.010) for {'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 3, 'n_estimators': 50}
0.016 (+/-0.010) for {'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 6, 'n_estimators': 10}
0.025 (+/-0.014) for {'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 6, 'n_estimators': 25}
0.017 (+/-0.009) for {'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 6, 'n_estimators': 50}
0.016 (+/-0.006) for {'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 9, 'n_estimators': 10}
0.013 (+/-0.009) for {'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 9, 'n_estimators': 25}
0.021 (+/-0.015) for {'max_depth': 3, 'max_features': 'sqrt', 'max_leaf_nodes': 9, 'n_estimators': 50}
0.015 (+/-0.004) for {'max_depth': 3, 'max_features': 'log2', 'max_leaf_n

#### Đánh giá mô hình

In [57]:
y_pred_grid = grid_search.predict(X_valid)

In [58]:
acc_train = metrics.accuracy_score(y_train, y_train_pred_rf)
acc_valid_grid = metrics.accuracy_score(y_valid, y_pred_grid)

print(f"Accuracy train split = {acc_train}")
print(f"Accuracy valid split = {acc_valid_grid}")

Accuracy train split = 1.0
Accuracy valid split = 0.0136986301369863


In [56]:
print(metrics.classification_report(y_pred_grid, y_valid))

              precision    recall  f1-score   support

       35311       0.00      0.00      0.00         0
       40000       0.00      0.00      0.00         0
       55993       0.00      0.00      0.00         0
       60000       0.00      0.00      0.00         0
       64500       0.00      0.00      0.00         0
       66500       0.00      0.00      0.00         0
       67000       0.00      0.00      0.00         0
       68400       0.00      0.00      0.00         0
       68500       0.00      0.00      0.00         0
       75000       0.00      0.00      0.00         0
       75500       0.00      0.00      0.00         0
       79500       0.00      0.00      0.00         0
       81000       0.00      0.00      0.00         0
       84500       0.00      0.00      0.00         0
       84900       0.00      0.00      0.00         0
       85000       0.00      0.00      0.00         0
       86000       0.00      0.00      0.00         0
       89471       0.00    